In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [65]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    d = vec - y
    u = d / np.linalg.norm(d)
    H = np.eye(u.shape[0]) - 2 * u * u.reshape(-1, 1)
    return y, H

Test your function using tests below:

In [191]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1, atol=2e-16)

In [62]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [124]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    H = np.eye(m)
    for i in range(n):
        x = a1[i:, i]
        _, h = householder(x)
        H1 = np.eye(m)
        H1[i:, i:] = h
        a1 = H1 @ a1
        H = H1 @ H
    Q = H.T
    R = a1
    return Q, R

In [105]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [125]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [126]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [138]:
np.isclose(abs(q), abs(qq))

array([[ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False],
       [ True,  True,  True, False, False]])

In [142]:
np.allclose(abs(r), abs(rr))

True

*Enter your explanation here* (10% of the total grade, peer-graded)

m > n (5 > 3) so it is economic QR, so 2 last columns of Q don't matter.  

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [1]:
def get_u_householder(vec):
    """Construct a Householder reflection vector to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    u : array of floats, shape (n,)
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    d = vec - y
    u = d / np.linalg.norm(d)
    return u


def qr_nomatrix(a):
    """Form QR decomposition of `a` via successive Householder reflections.
    
    Parameters
    ----------
    a : ndarray
        Input matrix
    
    Returns
    -------
    R : ndarray
        Upper triangular matrix of the QR decomposition
    
    U : ndarray
        Columns store successive Householder reflectors: `U[j:, j]` stores
        the Householder reflector for reducing the `j`-th column.
        
    See Also
    --------
    householder_apply : apply Householder reflectors stored in `U` to `a`.
    
    """
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    U = np.zeros_like(a1)
    for i in range(n):
        x = a1[i:, i]
        u = get_u_householder(x)
        U[i:, i] = u
        fix_a1 = np.zeros_like(a1)
        fix_a1[i:, i:] = -2 * u.reshape(-1, 1) * (u.reshape(1, -1) @ a1[i:, i:])
        a1 += fix_a1
    R = a1
    return R, U
    

    
def householder_apply(b, uu):
    """Apply the Householder reflectors stored in `uu` to `b`.
    
    The result is equivalent to
    >>> r, q = qr_decomp(a)
    >>> q.T @ b
    
    Parameters
    ----------
    b : ndarray
        Input matrix
    uu : ndarray
        Householder reflectors: `U[j:, j]` is the reflection vector
        for transforming the `j`-th column of `a`.
        
    Returns
    -------
    r : ndarray
        The result of applying the reflectors to `b`. Equivalent to
        computing `q.T @ b`.

    See Also
    --------
    qr_decomp
    
    """

    a1 = np.array(b, copy=True, dtype=float)
    m, n = a1.shape
    
    for i in range(n):
        u = uu[i:, i]
        fix_a1 = np.zeros_like(a1)
        fix_a1[i:, i:] = -2 * u.reshape(-1, 1) * (u.reshape(1, -1) @ a1[i:, i:])
        a1 += fix_a1
    R = a1
    return R

In [2]:
rndm = np.random.RandomState(1223)

a = rndm.uniform(size=(5, 5))
R1, U = qr_nomatrix(a)
R2 = householder_apply(a, U)
R_lib = qr(a)[1]
assert_allclose(R1, R2, atol=1e-14)


NameError: name 'np' is not defined